<a href="https://colab.research.google.com/github/elizabethavargas/Investing/blob/main/BasicInvestingGSPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Models for Continuously Investing in the S&P 500


In [1]:
# imports
import pandas as pd
import yfinance as yf

In [2]:
def get_stock_data(ticker, start, end):
    # Fetch historical data using yfinance
    sp500_data = yf.download(ticker, start=start, end=end)
    return sp500_data

In [3]:
# get data
sp500_data = get_stock_data('^GSPC', '2018-07-31', '2023-07-31') # S&P 500 index
sp500_data = pd.DataFrame(sp500_data)
sp500_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000
...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000


## Buy the Day
Invest every day the market is open

In [4]:
day_invest = 100

# initalize invested and total columns
sp500_data_day = sp500_data.copy()
sp500_data_day['Invested'] = 0
sp500_data_day['Total'] = 0
sp500_data_day['Return'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_day.iterrows():
  # invest
  num_shares += (day_invest / row['Close'])
  invested += day_invest
  sp500_data_day.at[index, 'Invested'] = invested
  sp500_data_day.at[index, 'Total'] = num_shares * row['Close']

# calculate return
sp500_data_day['Return'] = round((sp500_data_day['Total'] - sp500_data_day['Invested']) / sp500_data_day['Invested'], 3)
sp500_data_day

,Open,High,Low,Close,Adj Close,Volume,Invested,Total,Return
Date,,,,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000,100,100.000000,0.000
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000,200,199.895965,-0.001
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000,300,300.880741,0.003
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000,400,402.278087,0.006
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000,500,503.701438,0.007
...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,124900,163097.740506,0.306
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,125000,163582.310354,0.309
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,125100,162577.001659,0.300


## Little Dipper
Each Day, if close < previous close, buy at closing

In [5]:
def should_buy(stock_data, day_index):
    # Check if the stock dipped the previous day
    prev_close = stock_data.iloc[day_index - 1]['Close']
    today_close = stock_data.iloc[day_index]['Close']
    return today_close < prev_close

In [6]:
day_invest = 100

# True/False for buy
sp500_data_little = sp500_data.copy()
sp500_data_little['Buy'] = sp500_data_little.index.to_series().apply(lambda day: should_buy(sp500_data_little, sp500_data_little.index.get_loc(day)))

# initalize invested and total columns
sp500_data_little['Invested'] = 0
sp500_data_little['Total'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_little.iterrows():
  if row['Buy']:
    num_shares += (day_invest / row['Close'])
    invested += day_invest
    day_invest = 100
  else:
    day_invest += 100
  sp500_data_little.at[index, 'Invested'] = invested
  sp500_data_little.at[index, 'Total'] = num_shares * row['Close']

# calculate return
sp500_data_little['Return'] = round((sp500_data_little['Total'] - sp500_data_little['Invested']) / sp500_data_little['Invested'], 3)
sp500_data_little

,Open,High,Low,Close,Adj Close,Volume,Buy,Invested,Total,Return
Date,,,,,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000,True,100,100.000000,0.000
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000,True,200,199.895965,-0.001
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000,False,200,200.880741,0.004
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000,False,200,201.813668,0.009
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000,False,200,202.527731,0.013
...,...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,False,124700,162924.993089,0.307
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,False,124700,163309.155614,0.310
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,True,125100,162605.692598,0.300


## Big Dipper
Buy when the stock has dipped a certain amount


In [7]:
day_invest = 100
dip = .05
sp500_data_big = sp500_data.copy()
last_peak = sp500_data_big.iloc[0]['Close']

# initalize invested and total columns
sp500_data_big['Invested'] = 0
sp500_data_big['Total'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_big.iterrows():
  if row['Close'] <= last_peak*(1-dip):
    num_shares += (day_invest / row['Close'])
    invested += day_invest
    buy = True
    day_invest = 100
  else:
    buy = False
    day_invest += 100

  sp500_data_big.at[index, 'Buy'] = buy
  sp500_data_big.at[index, 'Invested'] = invested
  sp500_data_big.at[index, 'Total'] = num_shares * row['Close']

  if row['Close'] > last_peak:
    last_peak = row['Close']

# calculate return
sp500_data_big['Return'] = round((sp500_data_big['Total'] - sp500_data_big['Invested']) / sp500_data_big['Invested'], 3)
sp500_data_big

,Open,High,Low,Close,Adj Close,Volume,Invested,Total,Buy,Return
Date,,,,,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000,0,0.000000,False,NaN
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000,0,0.000000,False,NaN
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000,0,0.000000,False,NaN
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000,0,0.000000,False,NaN
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000,0,0.000000,False,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,124900,162775.825590,True,0.303
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,124900,163159.636391,False,0.306
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,125100,162257.183661,True,0.297


## Money Monday
Invest every monday

In [8]:
day_invest = 100

# initalize invested and total columns
sp500_data_mon = sp500_data.copy()
sp500_data['Invested'] = 0
sp500_data['Total'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_mon.iterrows():
  # invest
  if index.weekday() == 0:
    num_shares += (day_invest / row['Close'])
    invested += day_invest
    day_invest = 100
  else:
    day_invest += 100

  sp500_data_mon.at[index, 'Invested'] = invested
  sp500_data_mon.at[index, 'Total'] = num_shares * row['Close']

# calculate return
sp500_data_mon['Return'] = round((sp500_data_mon['Total'] - sp500_data_mon['Invested']) / sp500_data_mon['Invested'], 3)
sp500_data_mon

,Open,High,Low,Close,Adj Close,Volume,Invested,Total,Return
Date,,,,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000,0.0,0.000000,NaN
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000,0.0,0.000000,NaN
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000,0.0,0.000000,NaN
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000,0.0,0.000000,NaN
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000,500.0,500.000000,0.000
...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,124800.0,162987.577786,0.306
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,124800.0,163371.887880,0.309
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,124800.0,162268.000988,0.300
